# Lecture 3 Sector-coupling

## PV, CHP, Bio, Natural-gas, District heating

### 1)Import Packages

In [100]:
import pypsa
import pandapower as pp
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cartopy.crs as ccrs
import warnings
import subprocess
from shapely.errors import ShapelyDeprecationWarning
import logging

warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
logging.getLogger("pypsa.pf").setLevel(logging.WARNING)
plt.rc("figure", figsize=(10, 8))

In [101]:
os.chdir("C:\\Users\\par19744\\Python_projects\\practice notebooks\\PyPSA lectures BTU\\Lect_4")

### 2)Create a network and set Snapshots

In [102]:
# Create a new PyPSA network
network = pypsa.Network()
network.set_snapshots(range(1))  # Solve for a year 365*24
solver='glpk'

### 3)Add Buses

In [103]:
# Read excel file which contains Non renewable generators data

def add_buses(filename,header):
    try:
        bus_data = pd.read_csv(filename,header=header)
    except pd.errors.EmptyDataError:
        print("The CSV file is empty.")
        return
        
    for index,row in bus_data.iterrows():
        network.add(
            "Bus",
            name=row['bus'],
            v_nom=row['v_nom'],
            carrier=row['carrier'],
            )
    return network.buses

In [104]:
add_buses('buses.csv',0)

attribute,v_nom,type,x,y,carrier,unit,v_mag_pu_set,v_mag_pu_min,v_mag_pu_max,control,sub_network
Bus,,,,,,,,,,,
ele_bus,20.0,,0.0,0.0,AC,,1.0,0.0,inf,PQ,
heat_bus,1.0,,0.0,0.0,heat,,1.0,0.0,inf,PQ,
H2_bus,1.0,,0.0,0.0,hydrogen,,1.0,0.0,inf,PQ,
gas_bus,1.0,,0.0,0.0,gas,,1.0,0.0,inf,PQ,
biomass_bus,1.0,,0.0,0.0,biomass,,1.0,0.0,inf,PQ,


### 4)Add generators

In [105]:
def add_generators(filename,header):
    try:
        generators = pd.read_csv(filename, header=header)
    except pd.errors.EmptyDataError:
        print("The CSV file is empty.")
        return

    
    network.madd(
    "Generator",
        names= generators.generator,
        bus=list(generators.bus),
        carrier=list(generators.carrier),
        p_nom=list(generators.p_nom),
        p_nom_extendable=list(generators.p_nom_extendable),
        marginal_cost=list(generators.marginal_cost),
        efficiency=list(generators.efficiency)
    ) 
    return network.generators

In [106]:
add_generators('generators.csv',0)

,bus,carrier,p_nom,p_nom_extendable,marginal_cost,efficiency,control,type,p_nom_min,p_nom_max,p_min_pu,p_max_pu,p_set,q_set,sign,marginal_cost_quadratic,build_year,lifetime,capital_cost,committable,start_up_cost,shut_down_cost,stand_by_cost,min_up_time,min_down_time,up_time_before,down_time_before,ramp_limit_up,ramp_limit_down,ramp_limit_start_up,ramp_limit_shut_down,weight,p_nom_opt
Generator,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
coal,ele_bus,coal,50.0,False,50.0,0.35,PQ,,0.0,inf,0.0,1.0,0.0,0.0,1.0,0.0,0,inf,0.0,False,0.0,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
biomass,ele_bus,biomass,30.0,True,100.0,0.30,PQ,,0.0,inf,0.0,1.0,0.0,0.0,1.0,0.0,0,inf,0.0,False,0.0,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
wind,ele_bus,wind,40.0,False,0.0,NaN,PQ,,0.0,inf,0.0,1.0,0.0,0.0,1.0,0.0,0,inf,0.0,False,0.0,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0


### 5)Add load

In [107]:
# Read excel file which contains Non renewable generators data

def add_consumers(filename,header):
    try:
        load = pd.read_csv(filename, header=header)
    except pd.errors.EmptyDataError:
        print("The CSV file is empty.")
        return
        
    for index, row in load.iterrows():
        network.add(
            "Load",
            name=row['bus'],
            bus=row['bus'],
            p_set=row['load_MW'],
            carrier=row['carrier']
        )
    return network.loads

In [108]:
add_consumers('consumers.csv',0)

attribute,bus,carrier,type,p_set,q_set,sign
Load,,,,,,
ele_bus,ele_bus,AC,,120.0,0.0,-1.0
heat_bus,heat_bus,heat,,100.0,0.0,-1.0
H2_bus,H2_bus,hydrogen,,20.0,0.0,-1.0


### 6)Add links

In [109]:
links = pd.read_csv('links.csv', header=0)
links

,name,bus0,bus1,bus2,p_nom,p_nom_max,efficiency,efficiency2,capital_cost,p_nom_extendable
0,heat_pump,ele_bus,heat_bus,NaN,10,50,3.0,0.0,900,True
1,electrolyzer,ele_bus,H2_bus,NaN,10,50,0.8,0.0,150,True
2,gas_boiler,gas_bus,heat_bus,NaN,10,50,0.9,0.0,300,True
3,CHP,gas_bus,ele_bus,heat_bus,10,50,0.3,0.3,1400,True
4,OCGT,gas_bus,ele_bus,NaN,10,50,0.4,0.0,600,True


In [110]:
def add_links(filename, header):
    try:
        links = pd.read_csv(filename, header=header)
    except pd.errors.EmptyDataError:
        print("The CSV file is empty.")
        return
        
    for index, row in links.iterrows():
        network.add(
            "Link",
            name=row['name'],
            bus0=row['bus0'],
            bus1=row['bus1'],
            #bus2=row['bus2'],
            p_nom=row['p_nom'],
            p_nom_extendable=row['p_nom_extendable'],
            efficiency=row['efficiency'],
            #efficiency2=row['efficiency2'],
            capital_cost=row['capital_cost']
        )
    return network.links

In [111]:
add_links('links.csv',0)

attribute,bus0,bus1,type,carrier,efficiency,build_year,lifetime,p_nom,p_nom_extendable,p_nom_min,p_nom_max,p_set,p_min_pu,p_max_pu,capital_cost,marginal_cost,marginal_cost_quadratic,stand_by_cost,length,terrain_factor,committable,start_up_cost,shut_down_cost,min_up_time,min_down_time,up_time_before,down_time_before,ramp_limit_up,ramp_limit_down,ramp_limit_start_up,ramp_limit_shut_down,p_nom_opt
Link,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
heat_pump,ele_bus,heat_bus,,,3.0,0,inf,10.0,True,0.0,inf,0.0,0.0,1.0,900.0,0.0,0.0,0.0,0.0,1.0,False,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0
electrolyzer,ele_bus,H2_bus,,,0.8,0,inf,10.0,True,0.0,inf,0.0,0.0,1.0,150.0,0.0,0.0,0.0,0.0,1.0,False,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0
gas_boiler,gas_bus,heat_bus,,,0.9,0,inf,10.0,True,0.0,inf,0.0,0.0,1.0,300.0,0.0,0.0,0.0,0.0,1.0,False,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0
CHP,gas_bus,ele_bus,,,0.3,0,inf,10.0,True,0.0,inf,0.0,0.0,1.0,1400.0,0.0,0.0,0.0,0.0,1.0,False,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0
OCGT,gas_bus,ele_bus,,,0.4,0,inf,10.0,True,0.0,inf,0.0,0.0,1.0,600.0,0.0,0.0,0.0,0.0,1.0,False,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0


### 7)Add carriers

In [112]:
def add_carrier(filename, header):
    try:
        carrier = pd.read_csv(filename, header=header)
    except pd.errors.EmptyDataError:
        print("The CSV file is empty.")
        return
        
    for index, row in carrier.iterrows():
        network.add(
            "Carrier",
            name=row['carrier'],
            co2_emissions=row['co2_emissions'],
            nice_name=row['carrier']
        )
    return network.carriers

In [113]:
add_carrier('carrier.csv',0)

attribute,co2_emissions,color,nice_name,max_growth,max_relative_growth
Carrier,,,,,
gas,0.27,,gas,inf,0.0
coal,0.80,,coal,inf,0.0
wind,0.00,,wind,inf,0.0
biomass,0.00,,biomass,inf,0.0


### 8)Storages

def add_stores(filename, header):
    try:
        stores = pd.read_csv(filename, header=header)
    except pd.errors.EmptyDataError:
        print("The CSV file is empty.")
        return
        
    for index, row in stores.iterrows():
        network.add(
            "Store",
            name=row['name'],
            e_initial=row['e_initial'],
            e_nom=row['e_nom'],
            marginal_cost=row['marginal_cost'],
            bus=row['bus'],
            e_cyclic=row['e_cyclic'],
            e_nom_extendable=row['e_nom_extendable'],
        )
    return network.stores

add_stores('stores.csv',0)

### 9)Global constraints

In [114]:
network.add("GlobalConstraint", "co2_limit", sense="<=", constant=0.0)

In [115]:
network.lopf()

C:\Users\par19744\AppData\Local\Temp\ipykernel_6380\889609669.py:1: DeprecatedWarning:

lopf is deprecated as of 0.24 and will be removed in 1.0. Use linopy-based function ``n.optimize()`` instead. Migrate extra functionalities: https://pypsa.readthedocs.io/en/latest/examples/optimization-with-linopy-migrate-extra-functionalities.html.

INFO:pypsa.linopf:Prepare linear problem
INFO:pypsa.linopf:Total preparation time: 0.28s
INFO:pypsa.linopf:Solve linear problem using Glpk solver
INFO:pypsa.linopf:Optimization successful. Objective value: 1.41e+04


('ok', 'optimal')

In [1]:
# Your DataFrame df
df = pd.concat([network.generators_t.p.loc[0],
        network.links_t.p0.loc[0],
        network.loads_t.p.loc[0],
    ],
    keys=["Generators", "Links", "Line"],
    names=["Component", "index"],
).reset_index(name="Production")

# Plotting using PyPSA
fig, ax = plt.subplots(figsize=(13, 7))

# Plot generators
df_generators = df[df["Component"] == "Generators"]
ax.bar(df_generators["index"], df_generators["Production"], label="Generators")

# Plot links
df_links = df[df["Component"] == "Links"]
ax.bar(df_links["index"], df_links["Production"], label="Links")

# Plot loads
df_loads = df[df["Component"] == "Line"]
ax.bar(df_loads["index"], df_loads["Production"], label="Line")

# Customize the plot
ax.set_xlabel("Component")
ax.set_ylabel("Production (MW)")
ax.set_title("Power Production by Component")
ax.legend()

# Show the plot
plt.show()


NameError: name 'pd' is not defined